In [113]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
## Langmith tracking
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 7


In [114]:
import warnings

warnings.filterwarnings("ignore")
from pprint import pprint

from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language


In [115]:
loader = GenericLoader.from_filesystem(
    "./DotNetFramework/",
    glob="*",
    suffixes=[".cs"],
    parser=LanguageParser(language=Language.CSHARP, parser_threshold=1000),    
)
docs = loader.load()

In [116]:
docs[0].metadata['source']

'DotNetFramework/CustomWebControl.cs'

In [117]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

code_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.CSHARP, chunk_size=1000, chunk_overlap=0
)
documents = code_splitter.split_documents(docs)



In [118]:
documents

[Document(page_content='using System;\nusing System.Web.UI;\nusing System.Web.UI.WebControls;\n\nnamespace WebFormsApp\n{\n    public class CustomWebControl : WebControl\n    {\n        protected override void RenderContents(HtmlTextWriter writer)\n        {\n            writer.Write("Hello, this is a custom WebControl for .NET Framework WebForms.");\n        }\n    }\n\n    public partial class WebForm1 : Page\n    {\n        protected void Page_Load(object sender, EventArgs e)\n        {\n            CustomWebControl myControl = new CustomWebControl();\n            myControl.ID = "MyCustomControl";\n            this.form1.Controls.Add(myControl);\n        }\n    }\n}', metadata={'source': 'DotNetFramework/CustomWebControl.cs', 'language': <Language.CSHARP: 'csharp'>}),
 Document(page_content='using System;\nusing System.Data.SqlClient;\n\n\n    class DBManager\n    {\n        public static void ExecuteTopTenQuery()\n        {\n            string connectionString = "Data Source=Server

In [119]:
## FAISS Vector Database
from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [120]:
# from langchain_community.llms import Ollama
# llm=Ollama(model="llama2")

from langchain_openai import ChatOpenAI
# llm=ChatOpenAI(model="gpt-3.5-turbo")
llm=ChatOpenAI(model="gpt-4-turbo", temperature=0)


In [121]:
# ## Design ChatPrompt Template
# from langchain_core.prompts import ChatPromptTemplate

# prompt=ChatPromptTemplate.from_messages(
#     [
#         ("system","Hi ChatGPT, we are going to rewrite some old C# code which is based on .NET Framework. please rewrite the piece of code with .NET 5/6+ or ASP.NET Core."),
#         ("user","old code:{oldcode}")
#     ]
# )



## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Hi ChatGPT, we are going to rewrite some old C# code which is based on .NET Framework. 
Please follow below instructions to design your new code:
(1) New Code should be based on .NET 8+ or ASP.NET Core. 
(2) For any new source code file, make sure the file name has some relationship with original file name or class name for constancy by adding subffix ("Page" or "Service" or "Controller") to represent its function. 
(3) Outmost namespace is AspNetCoreApp                                          
(4) Generate the Startup.cs file 
(5) Use standard angle brackets (< and >) for generic types or HTML tag, DO NOT use the HTML-encoded versions (&lt; and &gt;) 
(6) Use Razor Pages page-based programming model to build web UI. Always create .cshtml and .cshtml.cs file for page implementation 
(7) For performance reason, if there are SQL query statements, continue use SQL query statement and don't convert to Entity Framework. But make sure the new code is .NET 8+ compatible
(8) If a program can run in .NET 8+ envionment without installation of .NET framework, don't change the code.
(9) Use Microsoft.Data.SqlClient instead of System.Data.SqlClient


Please follow below instructions to output response:
(1)Please use xml tag <description></description> to contain all your description; and use xml tag <NewCode><file name="xxxx"></file><file name="yyyy"></file>...</NewCode> to wrap source code files generated.
(2)The content within <file name="xxxx"></file> is valid C# code.
                                          
In addition, basedon the context provided, please help to identify.Net Framework features replaced.
Finally, list original files which can run in .NET 8+ environment without installation of .NET framework. 

<context>
{context}
</context>
Old Code: {input}"""
)


In [122]:
## Chain Introduction
## Create Stuff Document Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [123]:
"""A retriever is an interface that returns documents given an unstructured query. 
It is more general than a vector store. A retriever does not need to be able to store documents,
only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, 
but there are other types of retrievers as well.
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f6eaca9ff10>)

In [124]:
from langchain.chains import create_retrieval_chain

retriever_chain  = create_retrieval_chain(retriever, document_chain)

In [125]:
# response=retriever_chain.invoke({"input": documents[0].page_content})
response=retriever_chain.invoke({"input": ""})

In [126]:
response['answer']

'<description>\nThe provided context includes two main components from a .NET Framework application: a WebForms component and a database management class. The WebForms component uses System.Web.UI, which is not supported in .NET Core or .NET 8+. Therefore, it needs to be rewritten using Razor Pages, a more modern page-based programming model for building web UIs in ASP.NET Core.\n\nThe database management class uses System.Data.SqlClient for database operations, which should be replaced with Microsoft.Data.SqlClient to ensure compatibility with .NET 8+.\n\nBelow are the rewritten components following the provided instructions:\n</description>\n<NewCode>\n<file name="CustomWebControlPage.cshtml">\n@page\n@model AspNetCoreApp.Pages.CustomWebControlModel\n\n<h1>Hello, this is a custom WebControl for .NET 8+.</h1>\n</file>\n\n<file name="CustomWebControlPage.cshtml.cs">\nusing Microsoft.AspNetCore.Mvc.RazorPages;\n\nnamespace AspNetCoreApp.Pages\n{\n    public class CustomWebControlModel :

In [127]:
import re
from IPython.display import display, Markdown

# Your input string
input_string = response['answer']



code_match = re.search(r'<NewCode>\n(.*?)\n</NewCode>', input_string, re.DOTALL)
description_match = re.search(r'<description>\n(.*?)\n</description>', input_string, re.DOTALL)

# Extracted code and description
code = code_match.group(1) if code_match else None
description = description_match.group(1) if description_match else None

display(Markdown(description))

print(code)

<file name="CustomWebControlPage.cshtml">
@page
@model AspNetCoreApp.Pages.CustomWebControlModel

<h1>Hello, this is a custom WebControl for .NET 8+.</h1>
</file>

<file name="CustomWebControlPage.cshtml.cs">
using Microsoft.AspNetCore.Mvc.RazorPages;

namespace AspNetCoreApp.Pages
{
    public class CustomWebControlModel : PageModel
    {
        public void OnGet()
        {
        }
    }
}
</file>

<file name="DBManagerService.cs">
using System;
using Microsoft.Data.SqlClient;

namespace AspNetCoreApp.Services
{
    public static class DBManagerService
    {
        public static void ExecuteTopTenQuery()
        {
            string connectionString = "Data Source=ServerName;Initial Catalog=DatabaseName;Integrated Security=True";
            string query = "SELECT TOP 10 * FROM YourTable";

            using (SqlConnection connection = new SqlConnection(connectionString))
            {
                SqlCommand command = new SqlCommand(query, connection);
                try
   

The provided context includes two main components from a .NET Framework application: a WebForms component and a database management class. The WebForms component uses System.Web.UI, which is not supported in .NET Core or .NET 8+. Therefore, it needs to be rewritten using Razor Pages, a more modern page-based programming model for building web UIs in ASP.NET Core.

The database management class uses System.Data.SqlClient for database operations, which should be replaced with Microsoft.Data.SqlClient to ensure compatibility with .NET 8+.

Below are the rewritten components following the provided instructions:

In [128]:
pattern = re.compile(r'<file name="(.+?)">(.*?)</file>', re.DOTALL)

# Finding all matches
matches = pattern.findall(code)

# Loop through matches and write to files
for file_name, file_content in matches:
    # Strip leading and trailing whitespaces from the content
    file_content = file_content.strip()
    # Open the file in write mode and write the content
    with open("./DotNet/"+file_name, 'w') as file:
        file.write(file_content)
    print(f"File saved: {file_name}")

File saved: CustomWebControlPage.cshtml
File saved: CustomWebControlPage.cshtml.cs
File saved: DBManagerService.cs
File saved: Startup.cs
